In [1]:
import numpy as np
import matplotlib.pyplot as plt
import open3d as o3d
import os
import json

import sys
sys.path.append("/home/andang/workspace/Computational_Vision/HL2/Hololens-Project/hl2ss/viewer")

%matplotlib inline

Jupyter environment detected. Enabling Open3D WebVisualizer.
[Open3D INFO] WebRTC GUI backend enabled.
[Open3D INFO] WebRTCWindowSystem: HTTP handshake server disabled.


In [2]:
import cv2
import hl2ss
import hl2ss_3dcv

## Working on Aligning Mesh and Camera

### Code that author sent

In [137]:
# def get_calibration(host, calibration_path):
#     calibration_vlc = hl2ss_3dcv.get_calibration_rm(host, hl2ss.StreamPort.RM_VLC_LEFTFRONT, calibration_path)
#     calibration_lt  = hl2ss_3dcv.get_calibration_rm(host, hl2ss.StreamPort.RM_DEPTH_LONGTHROW, calibration_path)
    
#     uv2xy      = hl2ss_3dcv.compute_uv2xy(calibration_lt.intrinsics, hl2ss.Parameters_RM_DEPTH_LONGTHROW.WIDTH, hl2ss.Parameters_RM_DEPTH_LONGTHROW.HEIGHT)
#     xy1, scale = hl2ss_3dcv.rm_depth_compute_rays(uv2xy, calibration_lt.scale)

#     return calibration_vlc, calibration_lt, xy1, scale
def camera_to_image(intrinsics):
    return intrinsics

def block_to_list(points):
    return points.reshape((-1, points.shape[-1]))

def rm_vlc_to_rgb(image):
    return np.dstack((image, image, image))

def transform(points, transform4x4):
    return points @ transform4x4[:3, :3] + transform4x4[3, :3].reshape(([1] * (len(points.shape) - 1)).append(3))

def get_inhomogeneous_component(array):
    return array[..., 0:-1]

def get_homogeneous_component(array):
    return array[..., -1, np.newaxis]

def to_inhomogeneous(array):
    return get_inhomogeneous_component(array) / get_homogeneous_component(array)

def project(points, projection4x4):
    return to_inhomogeneous(transform(points, projection4x4))

def camera_to_rignode(extrinsics):
    return np.linalg.inv(extrinsics)

def reference_to_world(pose):
    return pose

def world_to_reference(pose):
    return np.linalg.inv(pose)

def rignode_to_camera(extrinsics):
    return extrinsics

def rm_depth_undistort(depth, undistort_map):
    return cv2.remap(depth, undistort_map[:, :, 0], undistort_map[:, :, 1], cv2.INTER_NEAREST)

def get_calibration():
    calib_pv = hl2ss_3dcv._load_calibration_pv(PATH+'../calibrations/personal_video')
    calib_lt = hl2ss_3dcv._load_calibration_rm_depth_longthrow(PATH+'../calibrations/rm_depth_longthrow')
    calib_rf = hl2ss_3dcv._load_calibration_rm_vlc(PATH+'../calibrations/rm_vlc_rightfront')
    calib_lf = hl2ss_3dcv._load_calibration_rm_vlc(PATH+'../calibrations/rm_vlc_leftfront')
    
    uv2xy = hl2ss_3dcv.compute_uv2xy(calib_lt.intrinsics, hl2ss.Parameters_RM_DEPTH_LONGTHROW.WIDTH, hl2ss.Parameters_RM_DEPTH_LONGTHROW.HEIGHT)
    xy1, scale = hl2ss_3dcv.rm_depth_compute_rays(uv2xy, calib_lt.scale)
    
    #return all calibrations + xy1 + scale
    return calib_pv, calib_lt, calib_lf, calib_rf, xy1, scale

def integrate(calibration_vlc, calibration_lt, data_vlc_payload, data_vlc_pose, data_lt_payload, data_lt_pose, rays, scale):
    # depth = rm_depth_undistort(data_lt_payload, calibration_lt.undistort_map) / scale
    depth = data_lt_payload / scale
    depth = depth[:, :, np.newaxis]
    # color = cv2.remap(data_vlc_payload, calibration_vlc.undistort_map[:, :, 0], calibration_vlc.undistort_map[:, :, 1], cv2.INTER_LINEAR)
    color = data_vlc_payload
    color = rm_vlc_to_rgb(color)

    # lt_points          = hl2ss_3dcv.rm_depth_to_points(rays, depth)
    lt_points = rays * depth
    
    lt_to_world        = camera_to_rignode(calibration_lt.extrinsics) @ reference_to_world(data_lt_pose)
    world_to_vlc_image = world_to_reference(data_vlc_pose) @ rignode_to_camera(calibration_vlc.extrinsics) @ camera_to_image(calibration_vlc.intrinsics)
    world_points       = transform(lt_points, lt_to_world)
    vlc_uv             = project(world_points, world_to_vlc_image)
    # color              = cv2.remap(color, vlc_uv[:, :, 0].astype(np.float32), vlc_uv[:, :, 1].astype(np.float32), cv2.INTER_LINEAR).astype(np.float32) / hl2ss._RANGEOF.U8_MAX
    pcd = o3d.geometry.PointCloud()
    pcd.points = o3d.utility.Vector3dVector(block_to_list(world_points))
    pcd.colors = o3d.utility.Vector3dVector(block_to_list(color))

    return pcd, world_points

### Calibration Retrieve Code

In [138]:
PATH = '/home/andang/workspace/Computational_Vision/HL2/Hololens-Project/datasets/john_place/slam/'
PATH_MESH = PATH + 'mesh/'

In [139]:
caliv_pv, calib_lt, calib_lf, calib_rf, xy, scale = get_calibration()

## VLC Longthrow Retrieve

In [140]:
with open(PATH+'vlc1_pose.json') as f: # vlc LF
    j_vlclf = json.load(f)
with open(PATH+'depth_lt_pose.json') as f: # long throw
    j_lt = json.load(f)

In [141]:
vlclf_tstamps = sorted(j_vlclf.keys())
lt_tstamps = sorted(j_lt.keys())

In [142]:
def get_im(tstamp, root_path):
    return cv2.imread(root_path+tstamp+'.png')

In [143]:
assert lt_tstamps[0] == vlclf_tstamps[0]

In [144]:
vlclf_im0 = np.mean(get_im(vlclf_tstamps[0], PATH+'vlc_lf/'),axis=2)
lt_im0 = np.mean(get_im(lt_tstamps[0], PATH+'depth_lt/'),axis=2)

In [150]:
# show the point cloud generation code
pcd0,pts3d = integrate(calib_lf, calib_lt, vlclf_im0, j_vlclf[vlclf_tstamps[0]], lt_im0, j_vlclf[vlclf_tstamps[0]], xy, scale)
pts3d = pts3d.T.reshape(3,-1)

xs = pts3d[0,:]
ys = pts3d[1,:]
zs = pts3d[2,:]

In [202]:
NUM_PCD = 1
sep = 1
pcds = []
for i in range(NUM_PCD):
    vlclf_im = np.mean(get_im(vlclf_tstamps[sep*i], PATH+'vlc_lf/'),axis=2)
    lt_im = np.mean(get_im(lt_tstamps[sep*i], PATH+'depth_lt/'),axis=2)
    pcd0,pts3d = integrate(calib_lf, calib_lt, vlclf_im, np.array(j_vlclf[vlclf_tstamps[sep*i]]), lt_im, np.array(j_vlclf[vlclf_tstamps[sep*i]]), xy, scale)
    pcds.append(pcd0)
    

TypeError: unsupported operand type(s) for *: 'open3d.cuda.pybind.geometry.PointCloud' and 'float'

### Mesh retrieve code

In [191]:
# loaded all mesh data structures
CAP_MESH = PATH + 'mesh/'
mesh_hmap = {}

for path in os.listdir(CAP_MESH):
    split_names = (path.split('.txt')[0]).split('_')
    info_type = split_names[1]
    timestamp = split_names[2]
    index = split_names[3]
    data = np.loadtxt(CAP_MESH+path) #load up info

    if not (timestamp in mesh_hmap):
        mesh_hmap[timestamp] = {}

    if not index in mesh_hmap[timestamp]:
        mesh_hmap[timestamp][index] = {}


    mesh_hmap[timestamp][index][info_type] = data

### adding Open3D Meshes for Visualizer

In [220]:
#open3d rendering
open3d_meshes = []
ctr = 0

length = 0
mu = np.zeros((4,1))
var = np.zeros((4,1))
for timestamp,j in mesh_hmap.items():
    for i,j in mesh_hmap[timestamp].items():
        vertex_positions = j['vertex'].copy()
        triangle_indices = j['triangles'].copy()
        pose = j['pose'].copy()
        vertex_position_scale = j['vertexscale'].copy()
        vertex_normals = j['normals'].copy()


        vertex_positions[:, 0:3] = vertex_positions[:, 0:3] * vertex_position_scale
        vertex_positions = (vertex_positions/vertex_positions[:,3:]) @ pose
        d = np.linalg.norm(vertex_normals, axis=1)
        vertex_normals[d > 0, :] = vertex_normals[d > 0, :] / d[d > 0, np.newaxis]
        vertex_normals = vertex_normals @ pose
        
        mu += vertex_positions.sum(axis=0).reshape(4,1)
        length += vertex_positions.shape[0]
        
        open3d_mesh = o3d.geometry.TriangleMesh()
        open3d_mesh.vertices = o3d.utility.Vector3dVector(vertex_positions[:,0:3])
        open3d_mesh.vertex_colors = o3d.utility.Vector3dVector(vertex_normals[:, 0:3])
        open3d_mesh.triangles = o3d.utility.Vector3iVector(triangle_indices)
        
        open3d_meshes.append(open3d_mesh)
        
        # ctr += 1
        # if ctr > 1:
        #     break
    break
    
mu /= length


for timestamp,j in mesh_hmap.items():
    for i,j in mesh_hmap[timestamp].items():    
        vertex_positions = j['vertex'].copy()
        triangle_indices = j['triangles'].copy()
        pose = j['pose'].copy()
        vertex_position_scale = j['vertexscale'].copy()
        vertex_normals = j['normals'].copy()


        vertex_positions[:, 0:3] = vertex_positions[:, 0:3] * vertex_position_scale
        vertex_positions = (vertex_positions/vertex_positions[:,3:]) @ pose
        d = np.linalg.norm(vertex_normals, axis=1)
        vertex_normals[d > 0, :] = vertex_normals[d > 0, :] / d[d > 0, np.newaxis]
        vertex_normals = vertex_normals @ pose
        
        var += ((vertex_positions.T - mu)**2).sum(axis=1).reshape(4,1)
var /= (length-1)

open3d_meshes.extend(pcds)

In [221]:
mu,np.sqrt(var)

(array([[  95.17896493],
        [ -95.18262423],
        [-138.89902316],
        [   1.        ]]),
 array([[ 907.98226169],
        [1493.24237951],
        [1507.30597984],
        [   0.        ]]))

In [201]:
o3d.visualization.draw_geometries(open3d_meshes, mesh_show_back_face=True,point_show_normal=True)

In [223]:
MORE_PATH = '/home/andang/workspace/Computational_Vision/HL2/Hololens-Project/datasets/john_place/'

In [226]:
mesh = o3d.io.read_triangle_mesh(MORE_PATH+"john_scene.obj")
o3d.visualization.draw_geometries(open3d_meshes+[mesh], mesh_show_back_face=True,point_show_normal=True)

In [232]:
pts = np.asarray(mesh.vertices).T

In [237]:
np.mean(pts,axis=1),np.var(pts,axis=1)

(array([ 1.05135688,  0.24483478, -2.78123832]),
 array([3.84721798, 0.6775916 , 5.86544716]))

In [238]:
np.min(pts,axis=1),np.max(pts,axis=1)

(array([-2.35313463, -0.81625557, -9.27962971]),
 array([5.39333105, 2.1154871 , 0.56640631]))